In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm

In [2]:
from vllm import SamplingParams, LLM

sampling_params = SamplingParams(temperature=0, max_tokens=8)
llm = LLM(model="meta-llama/Llama-2-7b-chat-hf")

INFO 02-19 17:55:30 llm_engine.py:73] Initializing an LLM engine with config: model='meta-llama/Llama-2-7b-chat-hf', tokenizer='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-19 17:56:31 llm_engine.py:223] # GPU blocks: 815, # CPU blocks: 512
INFO 02-19 17:56:33 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-19 17:56:39 model_runner.py:437] Graph capturing finished in 5 secs.


In [3]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
#                                             load_in_8bit=True, 
#                                             torch_dtype=torch.float16,
#                                             device_map='auto')

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [4]:
dataset = pd.read_csv('data/analysis_set.csv')

In [5]:
def fetch_job_data(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    return job_data[0]['title']


In [6]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [7]:
dataset['job_id'].drop_duplicates()

0         1373681
25        1373682
66        1373683
71        1373684
189       1373685
           ...   
133468    1376446
133579    1376758
134063    1375146
136145    1376759
136172    1376737
Name: job_id, Length: 2949, dtype: int64

In [8]:
dataset['user_id'].drop_duplicates()

0          439042
1         2015063
2         2777927
3           58130
4         2290933
           ...   
136201    2796578
136202    2679806
136235     986712
136245    1713212
136249    1463255
Name: user_id, Length: 30363, dtype: int64

In [9]:
# jt = "Assistant Manager - Tax & Regulatory Services (3-4 yrs)"
# rt = html.unescape(res_text)

In [10]:
input_prompt = f'''
<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
unstructured text of a user's resume, and the title of a job that they have applied to. Your task 
is to check if the resume contains work experience relevant to the job title being provided.
Respond with a JUST A SINGLE WORD answer "yes" or "no"[\INST]
This is the resume text:\n{{resume_text}}\n
This is the job title {{job_title}}\n
[INST]The answer is [\INST]: 
'''

In [11]:
input_prompt = input_prompt.format(resume_text=res_text,
                   job_title=jt)

NameError: name 'res_text' is not defined

In [12]:
model_input = tokenizer(input_prompt, return_tensors="pt").to("cuda")

NameError: name 'tokenizer' is not defined

In [13]:
start_time = time.time()
model.eval()
with torch.no_grad():
    full_document = tokenizer.decode(model.generate(**model_input, max_new_tokens=32)[0], skip_special_tokens=True)

print(f'Time taken : {time.time()-start_time}')

NameError: name 'model' is not defined

In [14]:
print(full_document)

NameError: name 'full_document' is not defined

In [140]:
def get_answer_from_vllm(resume_text, job_title, llm):

    #define the input prompt here
    input_prompt = f'''<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
    unstructured text of a user's resume, and the title of a job that they have applied to. Your task 
    is to check if the resume text contains the job title being provided.
    Respond with a JUST A SINGLE WORD answer "yes" or "no"[\INST]
    This is the job title {{job_title}}\n
    This is the resume text:\n{{resume_text}}\n
    The answer is : 
    '''

    input_prompt = input_prompt.format(resume_text=resume_text,
                   job_title=jt)
    
    outputs = llm.generate(input_prompt, sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text

In [141]:
dataset.loc[dataset['job_id']==1373685]['user_id'].tolist()

[2800600, 489341, 397888, 2775899, 165112, 2781750, 1281744]

In [142]:
job_user_list[:3]

[[1376416, 1286631], [1374541, 2045460], [1375720, 756906]]

In [143]:


# outputs = llm.generate(eval_prompt, sampling_params)
# generated_text = outputs[0].outputs[0].text


In [144]:
dataset.head(5)

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933


In [145]:
dataset.sample(frac=1)

,job_id,application_id,user_id
135099,1375322,144440115,1648845
4983,1373764,144279774,736061
100810,1376214,144378013,2065092
124366,1374996,144420252,2624716
109636,1373781,144393823,378151
...,...,...,...
53711,1374769,144259837,1668351
26537,1374088,144225573,2629026
91079,1375815,144360813,263995
25019,1374056,144201128,2401560


In [156]:
counts = pd.read_csv('data/apply_counts.csv')

In [162]:
counts.sort_values('apply_count').iloc[50:100]

,id,apply_count
2344,1376066,6
2345,1376054,6
2503,1376276,6
1197,1374885,6
433,1374115,6
2243,1375964,6
2667,1376313,6
2295,1376029,6
1578,1375272,6
2813,1376504,6


In [154]:
dataset.job_id.sample()

98490    1373869
Name: job_id, dtype: int64

In [155]:
dataset[dataset['job_id']==1373869].shape[0]

815

In [163]:
ds = dataset[dataset['job_id']==1376164]

In [164]:
ex = ds[['job_id','user_id']].to_numpy().tolist()

In [165]:
import random

In [166]:
responses = {}
# job_user_list = dataset[['job_id','user_id']].sample(frac=1).drop_duplicates().to_numpy().tolist()

jul = random.sample(job_user_list,50)

for jobid, userid in tqdm(ex):

    user_id_list = dataset.loc[dataset['job_id']==jobid]['user_id'].tolist()
    try:
        jt = fetch_job_data(str(jobid))
    except:
        continue

    if jobid in responses:
        pass
    else:
        responses[jobid] = {}

    try:
        es_output = get_user_info(userid)
        rt = es_output['resume'][0]
        rt = html.unescape(rt)
    except:
        continue
    
    full_doc = get_answer_from_vllm(rt,jt, llm)
    responses[jobid][userid] = full_doc

  0%|          | 0/5 [00:00<?, ?it/s]



Processed prompts:   0%|                                                                                                                             | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.56it/s]


Processed prompts:   0%|                                                                                                                             | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]


Processed prompts:   0%|                                                                                                                             | 0/1 [00:00<?, ?it/s]

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████

In [167]:
responses

{1376164: {1286631: ' yes',
  114972: ' Yes',
  294145: ' yes',
  2808438: ' yes ',
  58826: ' yes'}}